In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,56.89,72,72,4.70,broken clouds
1,1,Busselton,AU,-33.6500,115.3333,67.05,51,4,11.97,clear sky
2,2,Bethel,US,41.3712,-73.4140,46.49,68,1,3.44,clear sky
3,3,Thompson,CA,55.7435,-97.8558,35.46,87,100,4.56,overcast clouds
4,4,Bathsheba,BB,13.2167,-59.5167,81.21,74,40,13.80,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maxium temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maxium temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                177
City                   177
Country                177
Lat                    177
Lng                    177
Max Temp               177
Humidity               177
Cloudiness             177
Wind Speed             177
Current Description    177
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bathsheba,BB,13.2167,-59.5167,81.21,74,40,13.80,scattered clouds
9,9,Souillac,MU,-20.5167,57.5167,76.35,83,40,8.05,scattered clouds
13,13,Mahebourg,MU,-20.4081,57.7000,75.25,80,71,9.91,broken clouds
15,15,Kavaratti,IN,10.5669,72.6420,81.39,74,100,7.54,overcast clouds
22,22,Salalah,OM,17.0151,54.0924,75.29,73,0,4.61,clear sky
...,...,...,...,...,...,...,...,...,...,...
681,681,Jamame,SO,0.0722,42.7506,77.38,76,100,7.54,overcast clouds
683,683,Tomigusuku,JP,26.1850,127.6750,77.14,73,75,10.36,broken clouds
686,686,Lismore,AU,-28.8135,153.2773,80.49,69,100,12.66,light rain
689,689,Sembakung,ID,3.8167,117.2500,82.33,79,76,1.28,broken clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bathsheba,BB,81.21,scattered clouds,13.2167,-59.5167,
9,Souillac,MU,76.35,scattered clouds,-20.5167,57.5167,
13,Mahebourg,MU,75.25,broken clouds,-20.4081,57.7000,
15,Kavaratti,IN,81.39,overcast clouds,10.5669,72.6420,
22,Salalah,OM,75.29,clear sky,17.0151,54.0924,
23,Hilo,US,77.25,light rain,19.7297,-155.0900,
26,Port Blair,IN,82.87,overcast clouds,11.6667,92.7500,
29,Port-Gentil,GA,78.66,light rain,-0.7193,8.7815,
30,Atuona,PF,78.40,clear sky,-9.8000,-139.0333,
32,Boundiali,CI,75.49,clear sky,9.5217,-6.4869,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat}, {lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found..... skipping.")
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found..... skipping.
Hotel not found..... skipping.
Hotel not found..... skipping.
Hotel not found..... skipping.
Hotel not found..... skipping.
Hotel not found..... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
clean_hotel_df = hotel_df
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bathsheba,BB,81.21,scattered clouds,13.2167,-59.5167,Atlantis Hotel
9,Souillac,MU,76.35,scattered clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
13,Mahebourg,MU,75.25,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
15,Kavaratti,IN,81.39,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
22,Salalah,OM,75.29,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
23,Hilo,US,77.25,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
26,Port Blair,IN,82.87,overcast clouds,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"
29,Port-Gentil,GA,78.66,light rain,-0.7193,8.7815,Fengshui Residence
30,Atuona,PF,78.40,clear sky,-9.8000,-139.0333,Villa Enata
32,Boundiali,CI,75.49,clear sky,9.5217,-6.4869,Hotel Paysan Boundiali


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}  and  {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.1)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))